<a href="https://colab.research.google.com/github/ruth-chirinos/Internal_MIA/blob/main/HM_multitask_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Multi-task recommenders

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/recommenders/examples/multitask"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/multitask.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/recommenders/blob/main/docs/examples/multitask.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/recommenders/docs/examples/multitask.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In the [basic retrieval tutorial](basic_retrieval) we built a retrieval system using movie watches as positive interaction signals.

In many applications, however, there are multiple rich sources of feedback to draw upon. For example, an e-commerce site may record user visits to product pages (abundant, but relatively low signal), image clicks, adding to cart, and, finally, purchases. It may even record post-purchase signals such as reviews and returns.

Integrating all these different forms of feedback is critical to building systems that users love to use, and that do not optimize for any one metric at the expense of overall performance.

In addition, building a joint model for multiple tasks may produce better results than building a number of task-specific models. This is especially true where some data is abundant (for example, clicks), and some data is sparse (purchases, returns, manual reviews). In those scenarios, a joint model may be able to use representations learned from the abundant task to improve its predictions on the sparse task via a phenomenon known as [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning). For example, [this paper](https://openreview.net/pdf?id=SJxPVcSonN) shows that a model predicting explicit user ratings from sparse user surveys can be substantially improved by adding an auxiliary task that uses abundant click log data.

In this tutorial, we are going to build a multi-objective recommender for Movielens, using both implicit (movie watches) and explicit signals (ratings).

## Imports


Let's first get our imports out of the way.


In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [ ]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
import tensorflow_recommenders as tfrs
from sklearn.model_selection import train_test_split



# H&M Group


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import datetime


from sklearn.preprocessing import LabelEncoder

# Load datasets
#base_path_data_example = "/Users/rchirinos-macbook/Documents/Boss/MIA/TFM2023/PythonProject/SimilarProjects/HM-CreateDatasetSamples"
#base_path_global = "/Users/rchirinos-macbook/Documents/Boss/MIA/TFM2023/PythonProject/data"
transactions = pd.read_csv('transactions_train_sample00001.csv')
articles_total = pd.read_csv('articles.csv')
print('QTY of transactions '+str(len(transactions)))
# Merge datasets based on article_id
merged_df = pd.merge(transactions, articles_total, on='article_id', how='left')
print('QTY of merged '+str(len(merged_df)))

#customers    = pd.read_csv('customers_train_sample1.csv')
#articles     = pd.read_csv('articles.csv')

In [ ]:
merged_df.info()

In [ ]:
merged_df

In [ ]:
merged_df[['customer_id', 'article_id', 'prod_name']].sort_values(by=['article_id'])

In [ ]:
transactions_short = merged_df[['customer_id', 'article_id', 'prod_name', 'price']]
transactions_short.info()

In [ ]:
# Group together
transactions_groupped = transactions_short.groupby(['customer_id', 'article_id', 'prod_name']).sum().reset_index()
transactions_groupped

In [ ]:
def create_DataBinary(DataGrouped):
    transactions_binary = transactions_groupped.copy()
    transactions_binary['PurchasedYes'] = 1
    return transactions_binary

transactions_binary = create_DataBinary(transactions_groupped)
transactions_binary.head()

In [ ]:
transactions_purchased_data =transactions_binary.drop(['price', 'article_id'], axis=1)
print(transactions_purchased_data.info())

In [ ]:
transactions_purchased_data.head(5)

In [ ]:
# Obtener los valores únicos de article_id
articles_only_names= transactions_purchased_data[['prod_name']].drop_duplicates()
articles_only_names

In [ ]:
# Obtener la fecha y hora de inicio
fecha_inicio = datetime.datetime.now()
print("Fecha de inicio:", fecha_inicio)

# Crear un DataFrame con todas las combinaciones de customer_id y article_id
combinaciones = pd.merge(transactions_purchased_data['customer_id'].drop_duplicates(), articles_only_ids, how='cross')

# Obtener la fecha y hora de finalización
fecha_final = datetime.datetime.now()
print("Fecha de finalización:", fecha_final)
# Calcular la duración del proceso
duracion = fecha_final - fecha_inicio
print("Duración del proceso:", duracion)


In [ ]:
#Clean objects
articles_only_ids = None

In [ ]:
# Obtener la fecha y hora de inicio
fecha_inicio = datetime.datetime.now()
print("Fecha de inicio:", fecha_inicio)

# Combinar con las transacciones para obtener el DataFrame final
df_resultado = pd.merge(combinaciones, transactions_purchased_data, how='left', on=['customer_id', 'article_id'])

# Obtener la fecha y hora de finalización
fecha_final = datetime.datetime.now()
print("Fecha de finalización:", fecha_final)
# Calcular la duración del proceso
duracion = fecha_final - fecha_inicio
print("Duración del proceso:", duracion)


In [ ]:
#Clean objects
combinaciones = None
transactions_purchased_data = None

In [ ]:
# Obtener la fecha y hora de inicio
fecha_inicio = datetime.datetime.now()
print("Fecha de inicio:", fecha_inicio)

# Llenar los valores NaN con 0, ya que NaN indica que el cliente no compró ese artículo
df_resultado['PurchasedYes'].fillna(0, inplace=True)

# Obtener la fecha y hora de finalización
fecha_final = datetime.datetime.now()
print("Fecha de finalización:", fecha_final)
# Calcular la duración del proceso
duracion = fecha_final - fecha_inicio
print("Duración del proceso:", duracion)


## Preparing the dataset

We're going to use the Movielens 100K dataset.

In [ ]:
'''
ratings = tfds.load('movielens/100k-ratings', split="train")
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
})
movies = movies.map(lambda x: x["movie_title"])
# Randomly shuffle data and split between train and test.
train, test = train_test_split(dataset, test_size=0.2, random_state=42)


movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["customer_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))
0   customer_id   object
 1   article_id    int64
 2   PurchasedYes  float64

'''

And repeat our preparations for building vocabularies and splitting the data into a train and a test set:

In [ ]:

# Encode 'customer_id' to numerical values
label_encoder = LabelEncoder()
df_resultado['customer_id'] = label_encoder.fit_transform(df_resultado['customer_id'])



In [ ]:
df_resultado.info()

In [ ]:
# Assume 'PurchasedYes' is the target column
features = df_resultado.drop('PurchasedYes', axis=1)
labels = df_resultado['PurchasedYes']

In [ ]:
# Combine features and labels into a single dataset
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [ ]:
# Get the total size of the dataset
#dataset_size = len(list(dataset))
# Get the length of the dataset
dataset_size = tf.data.experimental.cardinality(dataset).numpy()
print(dataset_size)

In [ ]:
# Calculate the size for training and testing datasets
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

In [ ]:
# Split the dataset into training and testing datasets
train = dataset.take(train_size)
test = dataset.skip(train_size)


In [ ]:
# Optional: Shuffle the training dataset
train = train.shuffle(buffer_size=train_size, reshuffle_each_iteration=False)

In [ ]:
# Optional: Batch the datasets
batch_size = 32
train = train.batch(batch_size)
test = test.batch(batch_size)

In [ ]:
# Example: Print the first few elements of the training dataset
'''
#
for features, label in train.take(1):
    print("Training features:", features.numpy())
    print("Training labels:", label.numpy())

# Example: Print the first few elements of the testing dataset
for features, label in test.take(1):
    print("\nTesting features:", features.numpy())
    print("Testing labels:", label.numpy())

'''

In [ ]:
print(type(train))
print(type(test))

In [ ]:
train

In [ ]:
test

In [ ]:
unique_article_ids_int = df_resultado['article_id'].unique()


unique_article_ids = unique_article_ids_int.astype(int)
unique_customer_ids = df_resultado['customer_id'].unique()

In [ ]:
print(len(unique_article_ids))
print(type(unique_article_ids[0]))

In [ ]:
articles_total.info()

In [ ]:
'''
HELP TO IDENTIFY DUPLICATES
import pandas as pd

# Supongamos que tienes un DataFrame llamado 'df' con columnas 'article_id' y 'prod_name'
# df = ...

# Verificar si hay nombres de productos duplicados con diferentes article_id
duplicates = articles_total[articles_total.duplicated(subset=['prod_name'], keep=False)]

if duplicates.empty:
    print("No hay nombres de productos duplicados con diferentes article_id.")
else:
    print("Nombres de productos duplicados con diferentes article_id:")
    print(duplicates[['article_id', 'prod_name']])
'''

In [ ]:
# Create a dictionary for mapping IDs to names
id_to_name_mapping = dict(zip(articles_total['article_id'], articles_total['prod_name']))

# Get the first key-value pair
first_key, first_value = next(iter(id_to_name_mapping.items()))

# Print key and value
print("First key:", first_key)
print("First key Type:", type(first_key))
print("First value:", first_value)



# Use the mapping to get names for the given article IDs
unique_article_names = np.array([id_to_name_mapping[article_id] for article_id in unique_article_ids])

print(len(unique_article_names))
print(unique_article_names[0])

In [ ]:
unique_article_names = np.unique(unique_article_names)
print(unique_article_names[0])
print(len(unique_article_names))


In [ ]:
print(unique_customer_ids)
print(type(unique_customer_ids[0]))

In [ ]:
unique_customer_ids_str = unique_customer_ids.astype(str)
print(unique_customer_ids_str)
print(type(unique_customer_ids_str[0]))

## A multi-task model

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

In this tutorial, we will define our models as before, but instead of having  a single task, we will have two tasks: one that predicts ratings, and one that predicts movie watches.

The user and movie models are as before:

```python
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add 1 to account for the unknown token.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])
```

However, now we will have two tasks. The first is the rating task:

```python
tfrs.tasks.Ranking(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)
```

Its goal is to predict the ratings as accurately as possible.

The second is the retrieval task:

```python
tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128)
    )
)
```

As before, this task's goal is to predict which movies the user will or will not watch.

### Putting it together

We put it all together in a model class.

The new component here is that - since we have two tasks and two losses - we need to decide on how important each loss is. We can do this by giving each of the losses a weight, and treating these weights as hyperparameters. If we assign a large loss weight to the rating task, our model is going to focus on predicting ratings (but still use some information from the retrieval task); if we assign a large loss weight to the retrieval task, it will focus on retrieval instead.

In [ ]:
print(type(unique_article_names[0]))
print(type(unique_customer_ids[0]))

In [ ]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and movie models.
    self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_article_names, mask_token=None),
      tf.keras.layers.Embedding(len(unique_article_names) + 1, embedding_dimension)
    ])

    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_customer_ids_str, mask_token=None),
      tf.keras.layers.Embedding(len(unique_customer_ids) + 1, embedding_dimension)
    ])


    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            #candidates=movie.batch(128).map(self.movie_model)
            candidates=unique_article_ids
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["customer_id"])
    # And pick out the movie features and pass them into the movie model.
    movie_embeddings = self.movie_model(features["article_id"])

    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("PurchasedYes")

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [ ]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()
print(type(cached_train))
print(type(cached_test))

In [ ]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

The model does OK on predicting ratings (with an RMSE of around 1.11), but performs poorly at predicting which movies will be watched or not: its accuracy at 100 is almost 4 times worse than a model trained solely to predict watches.

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [ ]:
model = MovielensModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

We get the opposite result: a model that does well on retrieval, but poorly on predicting ratings.

### Joint model

Let's now train a model that assigns positive weights to both tasks.

In [ ]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

The result is a model that performs roughly as well on both tasks as each specialized model.

### Making prediction

We can use the trained multitask model to get trained user and movie embeddings, as well as the predicted rating:

In [ ]:
trained_movie_embeddings, trained_user_embeddings, predicted_rating = model({
      "user_id": np.array(["42"]),
      "movie_title": np.array(["Dances with Wolves (1990)"])
  })
print("Predicted rating:")
print(predicted_rating)

While the results here do not show a clear accuracy benefit from a joint model in this case, multi-task learning is in general an extremely useful tool. We can expect better results when we can transfer knowledge from a data-abundant task (such as clicks) to a closely related data-sparse task (such as purchases).